In [2]:
from Functions import *

In [3]:
data=pd.read_excel(r"../Data_and_Preprocessing/pm25_BC_corrected.xlsx")


In [4]:
device="cuda"

In [6]:
train=data.loc[data.label==0,:]
test=data.loc[data.label==1,:]


In [7]:
train_pm25=train.dropna(subset="pm25_Gaussion")
test_pm25=test.dropna(subset="pm25_Gaussion")
X_pm25_test,y_pm25_test=torch.from_numpy(test_pm25.loc[:,["corrected_week"]].values).to(device).T[0],torch.tensor(test_pm25.pm25_Gaussion.values).to(device)
X_pm25,y_pm25=torch.from_numpy(train_pm25.loc[:,["corrected_week"]].values).to(device).T[0],torch.tensor(train_pm25.pm25_Gaussion.values).to(device)
nuts_kernel,gpr_pm25=model(X_pm25,y_pm25,device) 
pyro_data_pm25,gpr_pm25=train_model(X_pm25,nuts_kernel,gpr_pm25,device,steps_num=2000) 


/home/michaelf/Pyro_periodic_submission/Seasonality_functions_periodic/Functions.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  rbf.period=torch.tensor(X.max())


Warmup [1]:   0%|          | 0/4000 [00:00, ?it/s]

Warmup [2]:   0%|          | 0/4000 [00:00, ?it/s]

Warmup [3]:   0%|          | 0/4000 [00:00, ?it/s]

Warmup [4]:   0%|          | 0/4000 [00:00, ?it/s]

KeyboardInterrupt: 

In [43]:
az.to_json(pyro_data_pm25, "Arviz_stats/mcmc_PM25_seasonality.json")
torch.save(gpr_pm25,"models/PM25_seasonality");

In [44]:
from sklearn.metrics import r2_score

r2_score(y_pm25_test.cpu().detach().numpy(),gpr_pm25(X_pm25_test)[0].cpu().detach().numpy())

-0.0009800254769605221

In [46]:
train_bc=train.dropna(subset="BC_Gaussion")
test_bc=test.dropna(subset="BC_Gaussion")
X_bc_test,y_bc_test=torch.from_numpy(test_bc.loc[:,["corrected_week"]].values).to(device).T[0],torch.tensor(test_bc.BC_Gaussion.values).to(device)
X_bc,y_bc=torch.from_numpy(train_bc.loc[:,["corrected_week"]].values).to(device).T[0],torch.tensor(train_bc.BC_Gaussion.values).to(device)

nuts_kernel_bc,gpr_bc=model(X_bc,y_bc,device,sigma_prior=0.1) 

/home/michaelf/Pyro_periodic/Seasonality_functions_periodic/Functions_periodic.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  rbf.period=torch.tensor(X.max())


In [47]:
pyro_data_bc,gpr_bc=train_model(X_bc,nuts_kernel_bc,gpr_bc,device,steps_num=2000) 
az.to_json(pyro_data_bc, "Arviz_stats/mcmc_BC_seasonality.json")
torch.save(gpr_bc,"models/BC_seasonality");

Warmup [1]:   0%|          | 0/4000 [00:00, ?it/s]

Warmup [2]:   0%|          | 0/4000 [00:00, ?it/s]

Warmup [3]:   0%|          | 0/4000 [00:00, ?it/s]

Warmup [4]:   0%|          | 0/4000 [00:00, ?it/s]

/data/michaelf/miniconda3/envs/gaussion_process/lib/python3.11/site-packages/arviz/data/io_pyro.py:158: UserWarning: Could not get vectorized trace, log_likelihood group will be omitted. Check your model vectorization or set log_likelihood=False
  warnings.warn(


In [50]:
az.summary(pyro_data_bc)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
kernel.lengthscale,1.987,0.912,0.721,3.838,0.014,0.014,4991.0,5445.0,1.0
kernel.variance,0.859,0.470,0.194,1.750,0.006,0.006,5895.0,5987.0,1.0
noise,0.265,0.013,0.241,0.291,0.000,0.000,5979.0,5355.0,1.0
